**Q1.** 

Query all columns for all American cities in the CITY table with populations larger than 100000.
The CountryCode for America is USA.
The CITY table is described as follows:

**CITY**

| **Field** | **Type** |
---|---
ID | NUMBER
NAME | VARCHAR2(17)
COUNTRYCODE | VARCHAR2(3)
DISTRICT | VARCHAR2(20)
POPULATION | NUMBER

**Q2.** 

Query the NAME field for all American cities in the CITY table with populations larger than 120000.
The CountryCode for America is USA.
The CITY table is described as follows:

**CITY**

| **Field** | **Type** |
---|---
ID | NUMBER
NAME | VARCHAR2(17)
COUNTRYCODE | VARCHAR2(3)
DISTRICT | VARCHAR2(20)
POPULATION | NUMBER

**Q3.**  
Query all columns (attributes) for every row in the CITY table.  
The CITY table is described as follows:

**CITY**

| **Field** | **Type** |
---|---
ID | NUMBER
NAME | VARCHAR2(17)
COUNTRYCODE | VARCHAR2(3)
DISTRICT | VARCHAR2(20)
POPULATION | NUMBER

**Q4.**  
Query all columns for a city in CITY with the ID 1661.  
The CITY table is described as follows:

**CITY**

| **Field** | **Type** |
---|---
ID | NUMBER
NAME | VARCHAR2(17)
COUNTRYCODE | VARCHAR2(3)
DISTRICT | VARCHAR2(20)
POPULATION | NUMBER

**Q5.**  
Query all attributes of every Japanese city in the CITY table. The COUNTRYCODE for Japan is JPN.  
The CITY table is described as follows:

**CITY**

| **Field** | **Type** |
---|---
ID | NUMBER
NAME | VARCHAR2(17)
COUNTRYCODE | VARCHAR2(3)
DISTRICT | VARCHAR2(20)
POPULATION | NUMBER

**Q6.**  
Query the names of all the Japanese cities in the CITY table. The COUNTRYCODE for Japan is JPN.  
The CITY table is described as follows:

**CITY**

| **Field** | **Type** |
---|---
ID | NUMBER
NAME | VARCHAR2(17)
COUNTRYCODE | VARCHAR2(3)
DISTRICT | VARCHAR2(20)
POPULATION | NUMBER

**Q7.**  
Query a list of CITY and STATE from the STATION table.  
The STATION table is described as follows:  
where LAT_N is the northern latitude and LONG_W is the western longitude.

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

**Q8.**  
Query a list of CITY names from STATION for cities that have an even ID number. Print the results in any order, but exclude duplicates from the answer.  
The STATION table is described as follows:  
where LAT_N is the northern latitude and LONG_W is the western longitude.

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

**Q9.**  
Find the difference between the total number of CITY entries in the table and the number of distinct CITY entries in the table.  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER


where LAT_N is the northern latitude and LONG_W is the western longitude.  
*For example, if there are three records in the table with CITY values 'New York', 'New York', 'Bengalaru', there are 2 different city names: 'New York' and 'Bengalaru'. The query returns 1, because total number of records - number of unique city names = 3 - 2 = 1.*


**Q10.**  
Query the two cities in STATION with the shortest and longest CITY names, as well as their respective lengths (i.e., number of characters in the name). If there is more than one smallest or largest city, choose the one that comes first when ordered alphabetically.  
The STATION table is described as follows: 

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.  
*Sample Input:*  
*For example, CITY has four entries: DEF, ABC, PQRS, and WXY.*  

*Sample Output:*  

| CITY | Length |
|------|--------|
| ABC  | 3      |
| PQRS | 4      |

**Hint:**  
*When ordered alphabetically, the CITY names are listed as ABC, DEF, PQRS, and WXY, with lengths and. The longest name is PQRS, but there are options for the shortest named city. Choose ABC, because it comes first alphabetically.*  

**Note:**  
*You can write two separate queries to get the desired output. It need not be a single query.*


**Q11.**  
Query the list of CITY names starting with vowels (i.e., a, e, i, o, or u) from STATION. Your result cannot contain duplicates.  
*Input Format:*  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q12.**  
Query the list of CITY names ending with vowels (a, e, i, o, u) from STATION. Your result cannot contain duplicates.  
*Input Format:*  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q13.**  
Query the list of CITY names from STATION that do not start with vowels. Your result cannot contain duplicates.  
*Input Format:*  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q14.**  
Query the list of CITY names from STATION that do not end with vowels. Your result cannot contain duplicates.  
*Input Format:*  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q15.**  
Query the list of CITY names from STATION that either do not start with vowels or do not end with vowels. Your result cannot contain duplicates.  
*Input Format:*  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.


**Q16.**  
Query the list of CITY names from STATION that do not start with vowels and do not end with vowels. Your result cannot contain duplicates.  
*Input Format:*  
The STATION table is described as follows:  

**STATION**

| **Field** | **Type** |
---|---
ID | NUMBER
CITY | VARCHAR2(21)
STATE | VARCHAR2(2)
LAT_N | NUMBER
LONG_W | NUMBER

where LAT_N is the northern latitude and LONG_W is the western longitude.
